In [1]:
# imports:
#   numpy for calculations
#   pandas for numerical manipulation
#   matplotlib for visualisation
#   os for getting and writing files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from IPython.display import display, clear_output, Markdown, Latex, HTML
from ipywidgets import widgets, interact
import os

## Stats Jupyter Notebook

### Welcome!

Hello Dad! (or whomever else has the displeasure of reading this)

### Code Overview

Below you will find a high-level overview of the code in this notebook. I will also endeavour to document as I go with comments and periodic Markdown cells, but we'll see how I go :S

#### Parameters

####


In [2]:
# fetch datasets
directory = 'datasets'
header = 'Execution result:'
subhead1 = 'Detected files:'
files = [f for f in os.listdir(directory) if os.path.isfile(
    os.path.join(directory, f))]


mdout = f'''
## {header}
### {subhead1}
`{files}`
'''

display(Markdown(mdout))


## Execution result:
### Detected files:
`['district-data.xlsx', 'PCT Data - Clean and Reduced.xlsx', 'sample_harvest1.csv', 'sample_harvest2.csv']`


In [3]:
# Helper functions

# Read file from string
def read_selection(file):
    if file.__contains__('xlsx'):
        return pd.read_excel('datasets/' + file)
    elif file.__contains__('csv'):
        return pd.read_csv('datasets/' + file)
    else:
        return pd.DataFrame()

# Filter dataset


def filter_dataframe(column, value):
    return file[file[column] == value]


def get_numeric():
    global filtered_set
    return filtered_set.select_dtypes(include='number')


def get_summary_df(df):
    col_sum = df.sum()
    col_avg = df.mean()

    summary_df = pd.DataFrame({
        'Sum': col_sum,
        'Average': col_avg
    })

    return summary_df.transpose()


def set_summary(**kwargs):
    disabled = kwargs.get('hidden')
    if disabled:
        summary_table.value = f'<p><h4> Data Summary hidden. </h4></p>'
    else:
        summary_table.value = f'''
        <p><h3> Data Summary: </h3></p>

        {get_summary_df(get_numeric()).to_html(max_rows=2)}

        '''


# UI Construction
# Create widgets
html_header = widgets.HTML(
    value="<h2>Data selection</h2><p>Select a dataset from the <b>datasets</b> directory.</p>"
)

file_select = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Dataset: '
)

file = read_selection(file_select.value)
filtered_set = file

summary_table = widgets.HTML(
    value=get_summary_df(get_numeric()).to_html(max_rows=2)
)


browser_toggle = widgets.Checkbox(
    value=True,
    description='Toggle Data Browser'
)

summary_toggle = widgets.Checkbox(
    value=True,
    description='Toggle Data Summary'
)


column_filter = widgets.Dropdown(options=file.columns, description='Column:')
value_filter = widgets.Dropdown(description='Value:')
reset_filter = widgets.Button(description='Reset Filters')


def update_values(*args):
    col = column_filter.value
    value_filter.options = file[col].unique()


update_values()


togglebox = widgets.VBox([browser_toggle, summary_toggle])
filterbox = widgets.HBox([column_filter, value_filter, reset_filter])

hbox = widgets.HBox([file_select, togglebox])

html_body = widgets.HTML(
    value=f'''<p>Selected set: <b>{file_select.value}</b>.</p>
    </p>Set columns: <i>{list(file.columns)}</i></p>'''
)


html_table = widgets.HTML(
    value=file.to_html(max_rows=10)
)


def on_select(selection):
    if selection['type'] == 'change' and selection['name'] == 'value':
        # update file
        global file
        file = read_selection(selection['new'])
        # update selection display
        html_body.value = f'''<p>Selected set: <b>{selection['new']}</b>.</p>
        </p>Set columns: <i>{list(file.columns)}</i></p>'''

        column_filter.options = file.columns
        update_values()
        html_table.value = file.to_html(max_rows=10)


def toggle_browser(*args):
    if browser_toggle.value:
        html_table.value = f'''
        <p><h3> Data Browser: </h3></p>

        {file.to_html(max_rows=10)}

        '''
        column_filter.layout.display = 'block'
        value_filter.layout.display = 'block'
        reset_filter.layout.display = 'block'
    else:
        html_table.value = f'<p><h4> Data Browser hidden. </h4></p>'
        column_filter.layout.display = 'none'
        value_filter.layout.display = 'none'
        reset_filter.layout.display = 'none'


def toggle_summary(*args):
    if summary_toggle.value:
        set_summary(hidden=False)
    else:
        set_summary(hidden=True)


def filter_table(*args):

    global filtered_set
    filtered_set = filter_dataframe(column_filter.value, value_filter.value)
    html_table.value = f'''
    <p><h3> Data Browser: </h3></p>

    {filtered_set.to_html(max_rows=10)}

    '''
    toggle_summary()


def filter_rst(*args):

    global filtered_set
    filtered_set = file

    html_table.value = f'''
    <p><h3> Data Browser: </h3></p>

    {filtered_set.to_html(max_rows=10)}

    '''
    toggle_summary()


# define and attach callbacks
file_select.observe(on_select, names='value')
browser_toggle.observe(toggle_browser, 'value')
summary_toggle.observe(toggle_summary, 'value')
column_filter.observe(update_values, 'value')
value_filter.observe(filter_table, 'value')
reset_filter.on_click(filter_rst)

# Format display boxes
menu = widgets.VBox([html_header, hbox, html_body,
                    html_table, filterbox, summary_table])

filtered_set = file

display(menu)

In [4]:
def get_size(df):
    rows, cols = df.shape
    sizestr = f'<b>{
        rows}</b> x <b>{cols}</b> (<i>{df.size} </i>total elements).'
    return sizestr


def get_selection_vals(df, col):
    return tuple(df[col].unique())


def on_selection(selection):
    if selection['type'] == 'change' and selection['name'] == 'value':
        vals = get_selection_vals(filtered_set, selection['new'])

        group_filter.options = vals
        group_filter.value = []


def final_filter(dataframe, group, val, exclude):
    final_dataset = pd.DataFrame({
        'Value': dataframe[val],
        'Group': dataframe[group]
    })

    final_dataset = final_dataset[~final_dataset['Group'].isin(
        exclude)]
    return final_dataset


def df_to_html_table(df):
    df = df.transpose()
    html = "<table style='border: 1px solid black; border-collapse: collapse;'>"
    # Add table headers
    html += "<tr>"
    for col in df.columns:
        html += f"<th style='border: 1px solid black; padding: 5px;'>{
            col}</th>"
    html += "</tr>"
    # Add table rows
    for i in range(len(df)):
        html += "<tr>"
        for col in df.columns:
            html += f"<td style='border: 1px solid black; padding: 5px;'>{
                df.iloc[i][col]}</td>"
        html += "</tr>"
    html += "</table>"
    return html


def update_table(request):
    global final_set
    final_set = final_filter(filtered_set, group_selection.value,
                             value_selection.value, group_filter.value)
    dataset_table.value = df_to_html_table(final_set)


output_area = widgets.Output()


def generate_plots(df):
    with output_area:
        clear_output(wait=True)
        final_dataset = df[~df['Group'].isin(
            group_filter.value)]

        model = ols('Value ~ Group', data=final_dataset).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)

        plt.figure(figsize=(20, 6))
        sns.boxplot(x='Group', y='Value', data=final_dataset)
        plt.title(group_selection.value + ' and ' +
                  value_selection.value + ' Boxplot')
        plt.xlabel(group_selection.value)
        plt.xticks(rotation=60)
        plt.ylabel(value_selection.value)
        plt.show()
        display(anova_table)


sns.set_theme(style="ticks", palette="pastel")

# display ANOVA selection criterion
margins = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    justify_content='space-between',  # Adjust this as needed
    padding='10px'
)

heading = widgets.HTML(
    value=f'''<p><h3>ANOVA Tool</h3>
    Loaded dataset: <b>{file_select.value}</b>.</p>
    <p>Size of filtered set: {get_size(filtered_set)} </p>'''
)

dataset_table = widgets.HTML(
    value=filtered_set.to_html(max_rows=4)
)

# Extract categories (category defined as any column containing string entries)
categories = [col for col in filtered_set.columns if filtered_set[col].apply(
    lambda x: isinstance(x, str)).all()]
categories.append('Month')

not_categories = [col for col in filtered_set.columns if col not in categories]

group_selection = widgets.Dropdown(
    options=categories,
    description='Group: ',
    layout={'width': 'max-content'}
)

value_selection = widgets.Dropdown(
    options=not_categories,
    description='Value: ',
    layout={'width': 'max-content'}
)


group_filter = widgets.SelectMultiple(
    options=get_selection_vals(filtered_set, group_selection.value),
    value=[],
    description='Exclude',
    disabled=False
)

final_set = final_filter(filtered_set, group_selection.value,
                         value_selection.value, group_filter.value)

generate_button = widgets.Button(description='Generate Plot')


def on_button_click(b):
    global final_set
    generate_plots(final_set)


generate_button.on_click(on_button_click)


group_selection.observe(on_selection, names='value')
group_selection.observe(update_table, 'value')
value_selection.observe(update_table, 'value')
group_filter.observe(update_table, 'value')

filter = widgets.HBox([group_selection, value_selection])
menu = widgets.VBox(
    [heading, filter, group_filter, generate_button], layout=margins)

display(menu)
display(dataset_table)
display(output_area)


# run stats on filtered set

KeyError: 'Month'

In [ ]:
# generate ANOVA from selection
final_dataset = pd.DataFrame({
    'Value': filtered_set[value_selection.value],
    'Group': filtered_set[group_selection.value]
})

final_dataset = final_dataset[~final_dataset['Group'].isin(
    group_filter.value)]

model = ols('Value ~ Group', data=final_dataset).fit()
anova_table = sm.stats.anova_lm(model, typ=2)


plt.figure(figsize=(20, 6))
sns.boxplot(x='Group', y='Value', data=final_dataset)
plt.title(group_selection.value + ' and ' + value_selection.value + ' Boxplot')
plt.xlabel(group_selection.value)
plt.xticks(rotation=60)
plt.ylabel(value_selection.value)
plt.show()

display(anova_table)